In [2]:
from keras import models
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Dropout
from keras.preprocessing import sequence
from keras.layers import LSTM
from keras.utils import np_utils

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from pandas import DataFrame, Series
import numpy as np

import re

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

import copy

Using TensorFlow backend.


In [21]:
rawdata = pd.read_csv("최종리스트_맥주 키워드 제거.csv",engine="python",encoding = "utf-8")
del rawdata["Unnamed: 0"]
morphs = list()

for i in range(0,len(rawdata)):
    morphs.append(list(rawdata.loc[i,:].dropna()))

In [22]:
len(rawdata)

24574

In [6]:
morphsVectored = list()


vocabulary = pd.read_csv("색인사전.csv",engine="python",encoding="utf-8")

vocabulary = vocabulary.to_dict(orient="records")[0]

for i in morphs:
    temporailyList = list()
    for k in i:
        #print(k)
        try:
            temporailyList.append(vocabulary[k])
        except KeyError:
            temporailyList.append(0)
    morphsVectored.append(temporailyList)


In [39]:
target = pd.read_csv("타겟.csv",engine="python",encoding="cp949")

In [40]:
target_encoded = target["0"]

In [41]:
len(morphsVectored)

24574

In [42]:
len(target_encoded)

24574

In [43]:
a = 0
b = 0

for i in range(0,len(morphsVectored)):
    a += len(morphsVectored[i])

mu = a/len(morphsVectored)

for i in range(0, len(morphsVectored)):
    b += (len(morphsVectored[i]) - mu)**2

std_dev = (b/len(morphsVectored))**0.5

In [44]:
print(mu,std_dev)

24.19158460161146 21.182734219409735


In [45]:
vectorized_seq = sequence.pad_sequences(morphsVectored,maxlen=50)

In [46]:
vectorized_seq

array([[   0,    0,    0, ..., 4820,  586, 3659],
       [   0,    0,    0, ..., 3976, 3976, 3117],
       [   0,    0,    0, ..., 4014, 1388, 3013],
       ...,
       [   0,    0,    0, ..., 3402, 2719, 4322],
       [   0,    0,    0, ..., 3434, 3862, 2790],
       [   0,    0,    0, ..., 1074,  745, 5288]])

In [47]:
X_train,X_test,y_train,y_test = train_test_split(vectorized_seq, target_encoded)

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [56]:
?EarlyStopping

In [57]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 10) # 조기종료 콜백함수 정의

In [61]:
model = Sequential()
model.add(Embedding(len(vocabulary)+1,128,input_length = 50))
model.add(Dropout(0.2))
model.add(Conv1D(256,3,padding="valid",activation="relu",strides=1))
model.add(MaxPooling1D(pool_size = 4))
#model.add(Conv1D(256,3,padding="valid",activation="relu",strides=1))
#model.add(MaxPooling1D(pool_size = 2))
model.add(LSTM(128))
model.add(Dense(13,activation="softmax"))

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [62]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 128)           746240    
_________________________________________________________________
dropout_3 (Dropout)          (None, 50, 128)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 48, 256)           98560     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 12, 256)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dense_3 (Dense)              (None, 13)                1677      
Total params: 1,043,597
Trainable params: 1,043,597
Non-trainable params: 0
_________________________________________________________________


In [63]:
model.fit(X_train,y_train,epochs=20,batch_size=64,validation_data=(X_test,y_test),callbacks=[early_stopping])

Train on 18430 samples, validate on 6144 samples
Epoch 1/20
18430/18430 [==============================] - 14s 738us/step - loss: 1.9170 - acc: 0.3516 - val_loss: 1.5177 - val_acc: 0.5135
Epoch 2/20
18430/18430 [==============================] - 12s 663us/step - loss: 1.3293 - acc: 0.5602 - val_loss: 1.4513 - val_acc: 0.5299
Epoch 3/20
18430/18430 [==============================] - 12s 662us/step - loss: 1.0682 - acc: 0.6467 - val_loss: 1.5151 - val_acc: 0.5286
Epoch 4/20
18430/18430 [==============================] - 12s 666us/step - loss: 0.8605 - acc: 0.7148 - val_loss: 1.6320 - val_acc: 0.5308
Epoch 5/20
18430/18430 [==============================] - 12s 667us/step - loss: 0.6730 - acc: 0.7809 - val_loss: 1.8405 - val_acc: 0.5158
Epoch 6/20
18430/18430 [==============================] - 12s 663us/step - loss: 0.5358 - acc: 0.8257 - val_loss: 2.0427 - val_acc: 0.5129
Epoch 7/20
18430/18430 [==============================] - 12s 664us/step - loss: 0.4265 - acc: 0.8621 - val_loss: 2.2

### 아래부턴 이래저래 연구중

In [ ]:
model = models.load_model("chatbot.model")

model.summary()
model.trainable = False
model.compile(loss = "categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [91]:
#model = load_model("chatbot.model")
#model.trainable = False

#소프트맥스층을 통과한 확률값이 아니라, LSTM에서 따끈따끈하게 출력된 값(의도벡터)를 얻기 위해, 마지막 dense층을 pop으로 삭제해준다.

model.pop()

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 128)           704896    
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 48, 256)           98560     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 12, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               197120    
Total params: 1,000,576
Trainable params: 1,000,576
Non-trainable params: 0
_________________________________________________________________


In [170]:
## for문을 돌려 X_train의 LSTM 의도벡터를 출력시킨다.

for i in range(0,len(X_train)):
    if i == 0:
        temp = model.predict(X_train[0:1])
    else :
        temp2 = model.predict(X_train[i:i+1])
        temp = np.vstack([temp,temp2])

In [328]:
id2word = {j : i for i,j in vocabulary.items()}

#워드임베딩층의 가중치를 가져온다.

embeddings = model.layers[0].get_weights()[0]

#색인 사전에 저장되어있는 색인화된 단어 목록을 가져와서 워드임베딩층의 해당 단어 벡터를 가져온다.

words_embeddings = {w:embeddings[idx+1] for w, idx in vocabulary.items()}

# 문장들이 색인화되어 저장되어있는 X_train에서 embedding층의 벡터화된 정보를 조회하여, 색인화된 문장을 벡터화된 문장으로 바꾼다.
test = dict()
test = {i : embeddings[i] for i in X_train[0]}

In [187]:
#트레인셋의 타겟 변수들
DataFrame(y_train).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [188]:
#출력된 확률값
round(pd.read_csv("확률행렬.csv",engine="python",encoding="utf-8")).head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [332]:
for j in le.classes_:
    for i in range(0,13):
        vector_cass = words_embeddings[j]
        vectors_res = np.array(vectors.loc[0,:]).reshape(-1,1)
        weights_res = weights.reshape(-1,1)
        weighted_vector = weights_res * vectors_res.T

        print(j,np.dot(vector_cass,weighted_vector[i])/(np.linalg.norm(weighted_vector[i])*np.linalg.norm(vector_cass)))

기네스 -0.06320143173693193
기네스 0.06320143173693195
기네스 0.06320143173693193
기네스 -0.06320143173693195
기네스 0.06320143173693196
기네스 0.06320143173693195
기네스 -0.06320143173693193
기네스 0.06320143173693195
기네스 -0.06320143173693193
기네스 0.06320143173693195
기네스 -0.06320143173693195
기네스 0.06320143173693193
기네스 0.06320143173693193
버드와이저 -0.04453096409086406
버드와이저 0.04453096409086407
버드와이저 0.044530964090864086
버드와이저 -0.044530964090864045
버드와이저 0.044530964090864066
버드와이저 0.04453096409086408
버드와이저 -0.044530964090864066
버드와이저 0.04453096409086406
버드와이저 -0.04453096409086406
버드와이저 0.04453096409086408
버드와이저 -0.04453096409086406
버드와이저 0.04453096409086406
버드와이저 0.04453096409086405
산미구엘 -0.07815855774891227
산미구엘 0.07815855774891228
산미구엘 0.07815855774891228
산미구엘 -0.07815855774891227
산미구엘 0.07815855774891226
산미구엘 0.0781585577489123
산미구엘 -0.07815855774891228
산미구엘 0.0781585577489123
산미구엘 -0.07815855774891228
산미구엘 0.07815855774891227
산미구엘 -0.07815855774891227
산미구엘 0.07815855774891227
산미구엘 0.07815855774891227
삿포로 -0.0

In [333]:
for j in le.classes_:
    for i in range(0,13):
        vector_cass = words_embeddings[j]
        vectors_res = np.array(vectors.loc[1,:]).reshape(-1,1)
        weights_res = weights.reshape(-1,1)
        weighted_vector = weights_res * vectors_res.T

        print(j,np.dot(vector_cass,weighted_vector[i])/(np.linalg.norm(weighted_vector[i])*np.linalg.norm(vector_cass)))

기네스 0.032264775376228634
기네스 -0.03226477537622864
기네스 -0.032264775376228655
기네스 0.03226477537622865
기네스 -0.03226477537622862
기네스 -0.03226477537622863
기네스 0.03226477537622865
기네스 -0.03226477537622865
기네스 0.032264775376228634
기네스 -0.032264775376228655
기네스 0.03226477537622865
기네스 -0.032264775376228655
기네스 -0.03226477537622865
버드와이저 -0.13863050390117634
버드와이저 0.13863050390117634
버드와이저 0.13863050390117634
버드와이저 -0.13863050390117634
버드와이저 0.13863050390117632
버드와이저 0.13863050390117637
버드와이저 -0.13863050390117632
버드와이저 0.13863050390117632
버드와이저 -0.13863050390117634
버드와이저 0.13863050390117634
버드와이저 -0.13863050390117634
버드와이저 0.13863050390117637
버드와이저 0.13863050390117637
산미구엘 -0.059247067509633246
산미구엘 0.059247067509633294
산미구엘 0.05924706750963326
산미구엘 -0.059247067509633274
산미구엘 0.05924706750963325
산미구엘 0.05924706750963327
산미구엘 -0.05924706750963327
산미구엘 0.05924706750963327
산미구엘 -0.059247067509633274
산미구엘 0.05924706750963327
산미구엘 -0.059247067509633274
산미구엘 0.059247067509633246
산미구엘 0.05924706750963

In [334]:
for j in le.classes_:
    for i in range(0,13):
        vector_cass = words_embeddings[j]
        vectors_res = np.array(vectors.loc[2,:]).reshape(-1,1)
        weights_res = weights.reshape(-1,1)
        weighted_vector = weights_res * vectors_res.T

        print(j,np.dot(vector_cass,weighted_vector[i])/(np.linalg.norm(weighted_vector[i])*np.linalg.norm(vector_cass)))

기네스 0.11379613907808604
기네스 -0.11379613907808599
기네스 -0.11379613907808603
기네스 0.11379613907808601
기네스 -0.11379613907808604
기네스 -0.11379613907808603
기네스 0.11379613907808601
기네스 -0.11379613907808606
기네스 0.11379613907808603
기네스 -0.11379613907808603
기네스 0.11379613907808601
기네스 -0.11379613907808603
기네스 -0.11379613907808603
버드와이저 0.12599107429702283
버드와이저 -0.12599107429702283
버드와이저 -0.12599107429702286
버드와이저 0.12599107429702286
버드와이저 -0.1259910742970229
버드와이저 -0.1259910742970229
버드와이저 0.12599107429702286
버드와이저 -0.12599107429702286
버드와이저 0.1259910742970229
버드와이저 -0.12599107429702283
버드와이저 0.12599107429702286
버드와이저 -0.1259910742970229
버드와이저 -0.12599107429702286
산미구엘 0.08350094686690848
산미구엘 -0.0835009468669085
산미구엘 -0.08350094686690848
산미구엘 0.08350094686690844
산미구엘 -0.08350094686690847
산미구엘 -0.08350094686690845
산미구엘 0.08350094686690845
산미구엘 -0.08350094686690848
산미구엘 0.08350094686690847
산미구엘 -0.08350094686690847
산미구엘 0.08350094686690844
산미구엘 -0.08350094686690847
산미구엘 -0.08350094686690848
삿포로 -0

In [350]:
good

array([ 0.0112435 ,  0.01626117, -0.02608678, -0.00465547, -0.01658903,
       -0.05096305, -0.00356845,  0.022622  , -0.01876216,  0.00818143,
       -0.00336798, -0.05094012, -0.05837329,  0.0047745 ,  0.04599338,
        0.05566986, -0.02716751, -0.02125545, -0.00525847,  0.09071777,
        0.02616188,  0.03651259,  0.03304788,  0.03063175,  0.02179771,
       -0.01772275, -0.05593007,  0.07067417, -0.0244693 , -0.01953671,
       -0.04648073, -0.0023631 ,  0.02196166, -0.00385395, -0.03950485,
       -0.0697339 , -0.02307081, -0.02674926, -0.01360447,  0.06189252,
       -0.03828458, -0.02737571,  0.01755727, -0.02691102,  0.03304856,
        0.01138357, -0.04143344, -0.02868658,  0.05254529,  0.01505004,
       -0.00438126,  0.05486773,  0.03050054,  0.04259002,  0.03420237,
       -0.0416372 , -0.02634784, -0.05134013,  0.00985894,  0.00774468,
        0.00473392, -0.02830628,  0.04002898,  0.03474801,  0.03297637,
       -0.02931027, -0.04132793,  0.06030182, -0.00323211, -0.00

In [351]:
merged = DataFrame(index=list(words_embeddings.keys()))
for i in list(words_embeddings.keys()):
    good = words_embeddings[i]
    good = DataFrame([good],index=[i])
    merged = pd.merge(merged,good,left_index=True,right_index=True,how="inner")


In [335]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)

merged_pca = pca.fit_transform(merged.T)

NameError: name 'merged' is not defined

In [575]:
## LSTM에서 출력된 의도벡터와 단어벡터의 코싸인 유사도를 구하는 함수

n = 0
df = DataFrame()
concat_cossine = DataFrame()

for i in le.classes_:
    vector_cass = words_embeddings[i]
    cossine = list()
    for j in range(0,len(vectors)):
        vector_intention = np.array(vectors.loc[j])
        cossine.append(np.dot(vector_cass,vector_intention)/(np.linalg.norm(vector_intention)*np.linalg.norm(vector_cass)))
    df = DataFrame(cossine,columns=[i])
    concat_cossine = pd.concat([df,concat_cossine],axis=1)

In [618]:
## 각 단어벡터(vector_cass)와 의도벡터(vector_intention)의 행렬 중 가장 코싸인 유사도가 높은 값들만 다시 가져와서 dict를 만든다.

ssi = list()

for i in range(len(concat_cossine)):
    where = {"value" : max(concat_cossine.loc[i,:]),"where" : np.where(concat_cossine.loc[i,:].values == max(concat_cossine.loc[i,:]))[0][0]}
    ssi.append(where)

## 각 단어들별로 코싸인 유사도가 가장 높은 타겟(맥주)값들의 평균값. 즉, 단어벡터(vector_cass)와 의도벡터(vector_intention)의 평균값을 본다.

for i in range(len(ssi)):
    n += ssi[i]["value"]

n/len(ssi)

##낮네
## 

0.1391058904944908